## Modules

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from modules import *

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Error loading punkt: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>


In [2]:
X,y=load_data('./data/labeledTrainData.tsv',colname=['review','sentiment'])

************** Loading Data ************


Summary of data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB
No of Rows: 25000
No of Columns: 3

Data View: Last 3 Instances

              id  sentiment                                             review
24997  "10905_3"          0  "Guy is a loser. Can't get girls, needs to bui...
24998  "10194_3"          0  "This 30 minute documentary Buñuel made in the...
24999   "8478_8"          1  "I saw this movie as a child and it broke my h...

Class Counts(label, row): Total
1    12500
0    12500
Name: sentiment, dtype: int64

Data View: First 5 Instances

         id  sentiment                                             review
0  "5814_8"   

In [3]:
## get the list of sentences
def review_to_sentences(review, tokenizer):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(raw_sentence)
    return sentences

sentences = []

# A tokenizer for dividing sentences
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
for review in X['review']:
    sentences+=review_to_sentences(review, tokenizer)
print(len(X))
print(len(sentences))

25000
266756


In [4]:
#preprocess
sentences = list(map(preprocess_data, sentences))

## word2vec

In [5]:
import time
from gensim.models import Word2Vec


# 训练模型
print("train word2vec...")
model = Word2Vec(sentences, size=250, window=5, min_count=5, workers=12, iter=10, sg=1)

train word2vec...


In [6]:
#save model
model_name = "./model/word2vec_train_data.model"
model.save(model_name)

## Get the vector of X_data

In [7]:
#get review vector
def get_review_vector(review):
    global word_vec
    word_vec = np.zeros((1,250))
    for word in review:
        if word in model:
            word_vec = np.concatenate((word_vec, np.array([model[word]])), axis=0)
    return pd.Series(word_vec.mean(axis = 0))

In [8]:
#preprocess
X_train, X_test, y_train,  y_test=split_data(X,y)
X_train=X_train.iloc[:, -1].apply(preprocess_data)
X_test=X_test.iloc[:, -1].apply(preprocess_data)


************** Spliting Data **************


************** Data After Splitting **************

Train Data: (20000, 1)
Test Data: (5000, 1)

Class Counts(label, row): Train
1    10018
0     9982
Name: sentiment, dtype: int64

Class Counts(label, row): Test
0    2518
1    2482
Name: sentiment, dtype: int64

First 5 Instance: Train
                                                  review
6167   "Some unsuspecting films carry a message that ...
3101   "Even the first 10 minutes of this movie were ...
17307  "To me this was more a wake up call, and reali...
3950   "Shower keeps within itself in so many ways. A...
893    "Brian Yuzna is often frowned upon as a direct...

First 5 Instance: Test
                                                  review
7799   "\"Girlfight\" is much more of a coming-of-age...
4427   "This movie will go down down in history as on...
14941  "I have to agree with Cal-37 it's a great movi...
11644  "Most of the Atomic Age monster movies I saw o...
15548  "I saw 

In [9]:
X_train_features = X_train.apply(get_review_vector)
X_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,0.040529,-0.056504,-0.119637,0.049388,-0.017562,-0.020660,0.039044,0.108896,-0.036914,-0.084274,...,0.018443,0.014037,-0.030989,-0.009136,-0.069813,0.069940,0.147117,-0.065478,0.146323,-0.083354
1,0.048778,-0.062611,-0.047675,0.070007,0.034960,0.010693,-0.035031,0.158397,-0.153992,-0.039895,...,-0.012425,-0.070851,0.006201,-0.001099,-0.078519,0.135354,0.141956,-0.037876,0.196699,0.003221
2,0.081458,-0.125897,-0.092605,0.073621,0.072933,0.002953,-0.006336,0.218053,-0.130141,-0.061667,...,0.029945,-0.053330,-0.002620,0.042959,-0.082090,0.137970,0.187883,-0.016886,0.167745,-0.046418
3,0.007783,-0.081884,-0.039946,0.030894,0.009550,0.052569,-0.003978,0.048011,-0.075120,-0.066794,...,-0.026188,-0.103521,0.000062,0.019256,-0.087347,0.117627,0.089923,-0.041807,0.135751,-0.038079
4,0.038180,-0.094914,-0.058401,-0.015450,0.041275,0.006204,-0.001451,0.169006,-0.066129,-0.105810,...,-0.010240,-0.040620,0.053438,-0.003202,-0.072592,0.123155,0.151914,0.000943,0.249972,-0.037866


In [10]:
X_test_features = X_test.apply(get_review_vector)

## Logistic regression

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# fit
LR_model = LogisticRegression()
LR_model = LR_model.fit(X_train_features, y_train)
y_predict = LR_model.predict(X_test_features)
y_test_np = y_test.values

# evaluate
accuracy = accuracy_score(y_test_np, y_predict)
print(f"accuracy: {accuracy}")

recall = recall_score(y_test_np, y_predict)
print(f"recall: {recall}")

precision = precision_score(y_test_np, y_predict)
print(f"precision: {precision}")

f1 = f1_score(y_test_np, y_predict)
print(f"f1: {f1}")

accuracy: 0.8648
recall: 0.8678485092667204
precision: 0.8609112709832134
f1: 0.8643659711075442


In [12]:
# use cross-validation to fit and evaluate evaluate the model
from sklearn.model_selection import cross_val_score
cross_val_model_LR = LogisticRegression()
scores = cross_val_score(cross_val_model_LR, pd.concat([X_train_features, X_test_features], axis=0), y_train.tolist()+y_test.tolist(), cv=10)
print("10 fold cross validation score of LogisticRegression:", np.mean(scores))

10 fold cross validation score of LogisticRegression: 0.8684000000000001


In [13]:
## test
test_data=pd.read_csv("data/testData.tsv",sep = "\t")
X_test=test_data['review'].apply(preprocess_data)
X_test_features = X_test.apply(get_review_vector)

In [14]:
test_predicted = np.array(LR_model.predict(X_test_features))

lr_output = pd.DataFrame(data=test_predicted, columns=['sentiment'])
lr_output['id'] = test_data['id']
lr_output = lr_output[['id', 'sentiment']]
lr_output.to_csv('out/word2vec.csv', index=False)